# Description
*author:* Vina My Pham<br>
*supervisor:* Robin van der Weide<br>
*project:* MSc internship project<br>
<br>
*date:* January 15 - July 19, 2024<br>
*host:* Kind group, Hubrecht Institute<br>
*university:* Bioinformatics, Wageningen University & Research<br>

---

**[flowchart: 00_flowchart.svg]**

Notebook to run pre-trained Cellpose models (v2.2.2) [1] on an image.<br>
Settings are written to a JSON file (".settings.json")

---

**References**<br>
1. Pachitariu, M., Stringer, C. Cellpose 2.0: how to train your own model. Nat Methods 19, 1634–1641 (2022). https://doi.org/10.1038/s41592-022-01663-4


# Notebook initialisation
This block contains the required code for running the notebook.<br>
Run this block before anything else.

## Select settings
- `mount_drive`: Whether to give access to files in your Drive.
- `pip_requirements_path`:
    - if mounted to the Drive, make sure the path starts with "/content/gdrive/MyDrive/"
- `use_gpu`: Whether to run Cellpose on the GPU.
    - To change hardware type: `Runtime` >> `Change runtime type` >> `Hardware accelerator`

In [ ]:
mount_drive = True #@param {type:"boolean"}
pip_requirements_path = "" #@param {type:"string"}
use_gpu = True #@param {type:"boolean"}

#@markdown [code: mounting drive, pip install, and module imports]
if mount_drive:
  from google.colab import drive
  drive.mount('/content/gdrive')

#installing cellpose and dependencies
import subprocess
subprocess.run(['apt-get', 'install', '-y', 'libcairo2-dev'], check=True)
subprocess.run(['pip', 'install', '-r', pip_requirements_path], check=True)
print("Succesfully installed requirements with pip")

#imports
import os
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np

from cellpose.io import imread
from cellpose import models
cellpose_models =  models.MODEL_NAMES
print("Succesfully installed requirements with pip")

#custom functions
def check_gpu_connection(use_gpu: bool) -> None:
    """Reports the details on GPU connection

    Args:
        use_gpu (bool): Whether to use the GPU for the script

    Returns:
        None
    """
    class GPUConnectionError(Exception):
        def __init__(self, message):
            self.message = message

    if not use_gpu:
        print("GPU will not be used in this run as `use_gpu` " +
              f"has been set to {use_gpu}")
        return None

    try:
        print("GPU connection requested:\n--------\n")
        subprocess.run(["nvidia-smi"], check=True)
        !nvidia-smi
    except FileNotFoundError:
        raise GPUConnectionError(f"`use_gpu` has been set to {use_gpu}. "+
               "However, the notebook is not connected to a GPU."+
               "\nPlease check the hardware type in the Colab Notebook "+
               "settings.")

    return None

def write_json(parameters: dict,
               save_dir: str,
               output_name: str,
               overwrite=False,
               verbose=True) -> str:
    """Write settings to a JSON file

    Args:
        parameters (dict): Settings to be written to the JSON file
        save_dir (str): The directory path where the JSON file will be saved
        overwrite (bool, optional): Overwrite if file exists. Default: False
        verbose (bool, optional): Print verbose. Default: True

    Returns:
        str: The absolute path of the generated JSON file

    Raises:
        FileExistsError: If a file in `save_dir` already
                         exists with `output_name` and `overwrite` is set to False
    """
    if os.path.exists(save_dir) == False:
      os.makedirs(save_dir)

    json_path = os.path.join(save_dir, output_name)

    if os.path.exists(json_path) and not overwrite:
        raise FileExistsError(f"File '{json_path}' exists and `overwrite` has" +
                              f" been set to {overwrite}")

    with open(json_path, 'w') as outfile_obj:
        json.dump(parameters, outfile_obj, indent=4)

    if verbose:
        print(f"All settings written to {json_path}")

    return json_path

def stacked(matrix):
    """Stack the channels of a image matrix

    Args:
        matrix (np.ndarray): image matrix (nChannels x nX x nY)
    Returns:
        np.ndarray: matrix representing the RGB format of an image (nX x nY)
    """
    return np.dstack((matrix[0,:,:], matrix[1,:,:], matrix[2,:,:]))

#check GPU connection
check_gpu_connection(True)

# Input files

In [ ]:
#@markdown **Provide the absolute path to the test slice.**
test_slice_path = "/content/gdrive/MyDrive/msc-internship_HI_2024_vmp/00_raw_data/nuclei_analysis/golden_standard/golden_standard_img.tif" #@param {type:"string"}
test_slice_path = os.path.join(test_slice_path, "")[:-1]

#@markdown **Provide the path to the main output directory**
output_dir = "/content/gdrive/MyDrive/msc-internship_HI_2024_vmp/02_results/01_run-modelzoo/run20240322" #@param {type:"string"}
output_dir = os.path.join(output_dir, "")

#@markdown [code: plot slice and channels]
test_slice = imread(test_slice_path) #shape: nChannels x nX x nY
print("Image has been stored in `test_slice`")

#plot channels and merged slice
nchannels = test_slice.shape[0]

y=7
figsize = (nchannels*y, y)
nplots = nchannels+1
rgb = {0:"Red", 1:"Green", 2:"Blue"}

fig, axes = plt.subplots(1, nplots, figsize=figsize)

for i in range(nplots-1):
    axes[i].imshow(test_slice[i,:,:], cmap=plt.cm.gray)
    axes[i].axis(False)
    axes[i].set_title(f"Channel {i+1} ({rgb[i]})")

axes[nplots-1].imshow(stacked(test_slice))
axes[nplots-1].axis(False)
axes[nplots-1].set_title("Merged");

# Cellpose model hyperparameters

In [ ]:
#@markdown **Specify model parameters.**
cyto_channel = 2 #@param {type:"number"}
nucleus_channel = 0 #@param {type:"number"}

diameter=30 #@param {type:"number"}
flow_threshold=0.4 #@param {type:"slider", min:0, max:1, step:0.01}
cellprob_threshold=0 #@param {type:"slider", min:0, max:1, step:0.01}

#code
channels = [cyto_channel,nucleus_channel]

# Write to JSON

In [ ]:
#@markdown [code: write to JSON]
parameters = {
    "input_files": {
        "test_slice_path" : test_slice_path,
        "test_slice_shape" : test_slice.shape
    },
    "cellpose_parameters": {
        "use_gpu" : use_gpu,
        "cyto_channel" : cyto_channel,
        "nucleus_channel" : nucleus_channel,
        "diameter" : diameter,
        "flow_threshold" : flow_threshold,
        "cellprob_threshold" : cellprob_threshold
    }
}

_ = write_json(parameters=parameters,
               save_dir=output_dir,
               output_name=".settings.JSON",
               overwrite=True,
               verbose=True)

# Running the Cellpose models

In [ ]:
#@markdown [code: running the models]
print(f"{datetime.datetime.now()}\tPre-trained models: {cellpose_models}")
if input("continue? (y/n)") != 'y':
    raise ValueError("Input was not `y`. Models will not run.")

#running .eval
for model_type in cellpose_models:
    print(f"{datetime.datetime.now()}\tRunning {model_type}")
    model = models.CellposeModel(gpu=use_gpu, model_type=model_type, net_avg = True)

    # run model
    masks, flows, styles = model.eval(test_slice,
                                    diameter=diameter,
                                    flow_threshold=flow_threshold,
                                    cellprob_threshold=cellprob_threshold,
                                    channels=channels)
    #store results
    filename = f"{output_dir}{model_type}_predictions.tiff"
    plt.imsave(filename, masks)

    print(f"{datetime.datetime.now()}\tFinished running {model_type}. Predictions stored under {filename}")